In [1]:
import os.path                      # OS処理
from collections import OrderedDict # 順序付き辞書(dict)

In [2]:
# 環境変数
root_path  = './test'            # 探索するパスの根 windows UNC path ("//host/computer/dir") を想定
search_file_name = 'keyfile.txt' # 探索するファイル名
target_file_path = './tmp'      # 保存先

In [3]:
# Windows のセパレータ'\\'への対応として、セパレータが '/' ではない場合は、 '/' を os.sep (OSのデフォルトセパレータ) に置き換える
if os.sep != '/':
    print('re')
    root_path = root_path.replace('/', os.sep)
    target_file_path = target_file_path.replace('/',os.sep)

In [4]:
# 指定のファイル名がリストに存在するかの判定
def isContain(filelist, keywoard):
    for filename in filelist:
        if filename == keywoard:
            return True

In [7]:
# 指定した path を巡回して、target_path_list を作る

#target_dict = OrderedDict() # 順序付き辞書(dict)
target_path_list = [] # 辞書入れリスト

for root, dirs, files in os.walk(root_path): # 相対path, サブディレクトリ, 内含ファイルリスト を走査

    #print( root,dirs,files) # debug
    
    if (isContain(files, search_file_name)): # 指定するファイル名を含むディレクトリの場合は以下を処理
        drive, path = os.path.splitdrive(root) # ネットワークドライブ名とパス名を分離         
        target_dict = {'network_drive': drive,                         # windows 共有ディレクトリのドライブ名
                       'path': os.path.join(path, search_file_name),   # 元ネタファイルの相対パス
                       'name': os.path.basename(path),                 # 最終ディレクトリ名を生成対象ファイル名に
                       'depth': path.count(os.sep)}                    # 階層の深さを
        target_path_list.append(target_dict)
    

In [8]:
target_path_list

[{'depth': 2,
  'name': 'FolderA',
  'network_drive': '',
  'path': './test/FolderA/keyfile.txt'},
 {'depth': 3,
  'name': 'FolderA1',
  'network_drive': '',
  'path': './test/FolderA/FolderA1/keyfile.txt'},
 {'depth': 3,
  'name': 'FolderA2',
  'network_drive': '',
  'path': './test/FolderA/FolderA2/keyfile.txt'},
 {'depth': 3,
  'name': 'FolderA3',
  'network_drive': '',
  'path': './test/FolderA/FolderA3/keyfile.txt'},
 {'depth': 3,
  'name': 'FolderA4',
  'network_drive': '',
  'path': './test/FolderA/FolderA4/keyfile.txt'},
 {'depth': 2,
  'name': 'FolderB',
  'network_drive': '',
  'path': './test/FolderB/keyfile.txt'},
 {'depth': 4,
  'name': 'FolderB11',
  'network_drive': '',
  'path': './test/FolderB/FolderB1/FolderB11/keyfile.txt'}]

In [ ]:
target_path_list[0]['network_drive']

これらの関数の多くは Windows の一律命名規則 (UNCパス名) を正しくサポートしていません。 splitunc() と ismount() は正しく UNC パス名を操作できます。

os.path.splitunc(path)(原文)
パス名 path をペア (unc, rest) に分割します。ここで unc は (r'\\host\mount' のような) UNC マウントポイント、そして rest は (r'\path\file.ext' のような) パスの残りの部分です。ドライブ名を含むパスでは常に unc が空文字列になります。

利用可能: Windows

os.path.splitunc(path)
Deprecated since version 3.1: Use splitdrive instead.


In [ ]:
os.sep

In [ ]:
for path in target_path_list:
    print (os.path.splitdrive(path[0]))